In [1]:

# https://cookbook.openai.com/examples/structured_outputs_intro
# https://cookbook.openai.com/examples/structured_outputs_multi_agent

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
import json
from openai import OpenAI
client = OpenAI()

In [4]:
MODEL = "gpt-4o-2024-08-06"

### Entity Extraction from user input
In this example, we will use function calling to search for products that match a user's preference based on the provided input.

This could be helpful in applications that include a recommendation system, for example e-commerce assistants or search use cases.

In [8]:
product_search_prompt = '''
    You are a clothes recommendation agent, specialized in finding the perfect match for a user.
    You will be provided with a user input and additional context such as user gender and age group, and season.
    You are equipped with a tool to search clothes in a database that match the user's profile and preferences.
    Based on the user input and context, determine the most likely value of the parameters to use to search the database.
    
    Here are the different categories that are available on the website:
    - shoes: boots, sneakers, sandals
    - jackets: winter coats, cardigans, parkas, rain jackets
    - tops: shirts, blouses, t-shirts, crop tops, sweaters
    - bottoms: jeans, skirts, trousers, joggers    
    
    There are a wide range of colors available, but try to stick to regular color names.
'''

product_search_function = {
    "type": "function",
    "function": {
        "name": "product_search",
        "description": "Search for a match in the product database",
        "parameters": {
            "type": "object",
            "properties": {
                "category": {
                    "type": "string",
                    "description": "The broad category of the product",
                    "enum": ["shoes", "jackets", "tops", "bottoms"]
                },
                "subcategory": {
                    "type": "string",
                    "description": "The sub category of the product, within the broader category",
                },
                "color": {
                    "type": "string",
                    "description": "The color of the product",
                },      
            },
            "required": ["category", "subcategory", "color"],
            "additionalProperties": False,
        }
    },
    "strict": True
}

def get_response(user_input, context):
    response = client.chat.completions.create(
        model=MODEL,
        temperature=0,
        messages=[
            {
                "role": "system",
                "content": product_search_prompt
            },
            {
                "role": "user",
                "content": f"CONTEXT: {context}\n USER INPUT: {user_input}"
            }
        ],
        tools=[product_search_function]
    )

    return response.choices[0].message.tool_calls

In [9]:
example_inputs = [
    {
        "user_input": "I'm looking for a new coat. I'm always cold so please something warm! Ideally something that matches my eyes.",
        "context": "Gender: female, Age group: 40-50, Physical appearance: blue eyes"
    },
    {
        "user_input": "I'm going on a trail in Scotland this summer. It's goind to be rainy. Help me find something.",
        "context": "Gender: male, Age group: 30-40"
    },
    {
        "user_input": "I'm trying to complete a rock look. I'm missing shoes. Any suggestions?",
        "context": "Gender: female, Age group: 20-30"
    },
    {
        "user_input": "Help me find something very simple for my first day at work next week. Something casual and neutral.",
        "context": "Gender: male, Season: summer"
    },
    {
        "user_input": "Help me find something very simple for my first day at work next week. Something casual and neutral.",
        "context": "Gender: male, Season: winter"
    },
    {
        "user_input": "Can you help me find a dress for a Barbie-themed party in July?",
        "context": "Gender: female, Age group: 20-30"
    }
]

In [10]:
def print_tool_call(user_input, context, tool_call):
    args = tool_call[0].function.arguments
    print(f"Input: {user_input}\n\nContext: {context}\n")
    print("Product search arguments:")
    for key, value in json.loads(args).items():
        print(f"{key}: '{value}'")
    print("\n\n")

In [11]:
for ex in example_inputs:
    ex['result'] = get_response(ex['user_input'], ex['context'])

In [12]:
for ex in example_inputs:
    print_tool_call(ex['user_input'], ex['context'], ex['result'])

Input: I'm looking for a new coat. I'm always cold so please something warm! Ideally something that matches my eyes.

Context: Gender: female, Age group: 40-50, Physical appearance: blue eyes

Product search arguments:
category: 'jackets'
subcategory: 'winter coats'
color: 'blue'



Input: I'm going on a trail in Scotland this summer. It's goind to be rainy. Help me find something.

Context: Gender: male, Age group: 30-40

Product search arguments:
category: 'jackets'
subcategory: 'rain jackets'
color: 'blue'



Input: I'm trying to complete a rock look. I'm missing shoes. Any suggestions?

Context: Gender: female, Age group: 20-30

Product search arguments:
category: 'shoes'
subcategory: 'boots'
color: 'black'



Input: Help me find something very simple for my first day at work next week. Something casual and neutral.

Context: Gender: male, Season: summer

Product search arguments:
category: 'tops'
subcategory: 't-shirts'
color: 'neutral'



Input: Help me find something very simple